# Experimental pipeline for GEQIE

## Setup

### Imports

In [1]:
import json
import subprocess
import tempfile

import pandas as pd

from concurrent import futures
from pathlib import Path
from typing import Any, Dict

In [2]:
%load_ext ipyform
%form_config --auto-detect 1

### Configurations

In [3]:
# @markdown #### Input and output paths
input_path = "../assets/test_images/grayscale" # @param {type:"string"}
output_path = "../.outputs"                    # @param {type:"string"}


# @markdown #### Encoding 
encoding_method = "frqi"           # @param {type:"string"}
grayscale = True                   # @param {type:"boolean"}
n_shots = 1024                     # @param {type:"integer"}
return_padded_counts = True        # @param {type:"boolean"}

# @markdown #### Concurrency config
max_concurrency = None             # @param {type:"integer"}
# @markdown **NOTE:** Type `None` for default value of `max_concurrency` (number of all available system CPUs)

input_path = Path(input_path)

FormWidget(children=(VBox(children=(HTML(value=''), HTML(value='<h4>Input and output paths</h4>'), Box(childre…

### Processing functions

In [5]:
def process_file(
        input_path: Path, 
        encoding_method: str, 
        n_shots: int, 
        grayscale: bool,
        return_padded_counts: bool,
        workdir: str,
) -> Dict[str, Any]:
    with tempfile.TemporaryFile(dir=workdir, prefix=f"{input_path.name}.", suffix=".json", mode="w", delete=False) as output_file:
        process_result = subprocess.run(
            f"geqie simulate \
                --encoding {encoding_method} \
                --image-path {input_path} \
                --n-shots {n_shots} \
                --return-padded-counts {return_padded_counts} \
                --grayscale {grayscale} \
                --output-path {output_file.name} \
            ".replace("  ", ""),
        )

    if process_result.returncode:
        raise RuntimeError(process_result.stderr.decode("utf-8"))

    result = json.loads(Path(output_file.name).read_text())

    return pd.DataFrame([{"name": ".".join(output_file.name.split(".")[:2]), "result": result}])


def process_directory(
        input_path: Path, 
        encoding_method: str, 
        n_shots: int, 
        grayscale: bool,
        return_padded_counts: bool, 
        workdir: str,
) -> Dict[str, Any]:
    workdir_path = Path(workdir)
    input_files = input_path.glob("*")

    with futures.ThreadPoolExecutor() as executor:
        execute_futures = [
            executor.submit(
                process_file,
                input_file_path,
                encoding_method,
                n_shots,
                grayscale,
                return_padded_counts,
                workdir,
            ) for input_file_path in input_files
        ]

    for f in execute_futures:
        f.result()

    dfs = []
    for temp_file_path in workdir_path.glob("*"):
        result = json.loads(temp_file_path.read_text())
        dfs.append(pd.DataFrame(
            [{"name": ".".join(temp_file_path.name.split(".")[:2]), "result": result}]
        ))
    
    return pd.concat(dfs)


### Run

In [6]:
def run():
    with tempfile.TemporaryDirectory() as workdir:
        if input_path.is_file():
            df = process_file(
                input_path=input_path,
                encoding_method=encoding_method,
                n_shots=n_shots,
                return_padded_counts=return_padded_counts,
                grayscale=grayscale,
                workdir=workdir,
            )
        if input_path.is_dir():
            df = process_directory(
                input_path=input_path,
                encoding_method=encoding_method,
                n_shots=n_shots,
                return_padded_counts=return_padded_counts,
                grayscale=grayscale,
                workdir=workdir,
            )

    print(df)

run()

                  name                                             result
0       full_white.png  {'000': 0, '001': 260, '010': 0, '011': 220, '...
0         test_2x4.png  {'00000': 0, '00001': 67, '00010': 0, '00011':...
0    test_flag_4x4.png  {'00000': 0, '00001': 61, '00010': 0, '00011':...
0       test_image.png  {'000': 0, '001': 266, '010': 42, '011': 219, ...
0   test_image0xf0.png  {'000': 146, '001': 105, '010': 123, '011': 12...
0  test_image_1010.png  {'000': 0, '001': 274, '010': 79, '011': 194, ...
0   test_image_4x4.png  {'00000': 0, '00001': 54, '00010': 59, '00011'...
